In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score

In [3]:
df = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/train.csv')

In [4]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'],inplace=True)

In [6]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [7]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(df.drop(columns=['Survived'],axis=1),df['Survived'],test_size=0.2)

In [10]:
x_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
834,3,male,18.0,0,0,8.3000,S
90,3,male,29.0,0,0,8.0500,S
725,3,male,20.0,0,0,8.6625,S
154,3,male,NaN,0,0,7.3125,S
221,2,male,27.0,0,0,13.0000,S


In [11]:
y_train

834    0
90     0
725    0
154    0
221    0
      ..
863    0
626    0
349    0
326    0
819    0
Name: Survived, Length: 712, dtype: int64

In [12]:
si_age= SimpleImputer()
si_embarked = SimpleImputer(strategy='most_frequent')

In [13]:
x_train_age = si_age.fit_transform(x_train[['Age']])
x_train_embarked = si_embarked.fit_transform(x_train[['Embarked']])

x_test_age = si_age.fit_transform(x_test[['Age']])
x_test_embarked = si_embarked.fit_transform(x_test[['Embarked']])

In [14]:
ohe_sex = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_age = OneHotEncoder(sparse=False,handle_unknown='ignore')
ohe_embarked = OneHotEncoder(sparse=False,handle_unknown='ignore')

x_train_sex = ohe_sex.fit_transform(x_train[['Sex']])
x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)

x_test_sex = ohe_sex.transform(x_test[['Sex']])
x_test_embarked = ohe_embarked.transform(x_test_embarked)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [15]:
x_train

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
834,3,male,18.0,0,0,8.3000,S
90,3,male,29.0,0,0,8.0500,S
725,3,male,20.0,0,0,8.6625,S
154,3,male,NaN,0,0,7.3125,S
221,2,male,27.0,0,0,13.0000,S
...,...,...,...,...,...,...,...
863,3,female,NaN,8,2,69.5500,S
626,2,male,57.0,0,0,12.3500,Q
349,3,male,42.0,0,0,8.6625,S
326,3,male,61.0,0,0,6.2375,S


In [16]:
x_train_embarked

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [17]:
 x_train_rem = x_train.drop(columns=['Sex','Age','Embarked'])
 x_test_rem = x_test.drop(columns=['Sex','Age','Embarked'])

In [18]:
import numpy as np
x_train_transfformed = np.concatenate((x_train_rem,x_train_age,x_train_sex,x_train_embarked),axis=1)
x_test_transfformed = np.concatenate((x_test_rem,x_test_age,x_test_sex,x_test_embarked),axis=1)

In [19]:
clf = DecisionTreeClassifier()
clf.fit(x_train_transfformed,y_train)

DecisionTreeClassifier()

In [20]:
y_pred = clf.predict(x_test_transfformed)

In [21]:
y_pred

array([0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 0])

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.7653631284916201

In [23]:
import pickle

In [24]:
pickle.dump(ohe_sex,open('/content/drive/MyDrive/ColabNotebooks/ohe_sex.pkl','wb'))
pickle.dump(ohe_age,open('/content/drive/MyDrive/ColabNotebooks/ohe_age.pkl','wb'))
pickle.dump(clf,open('/content/drive/MyDrive/ColabNotebooks/clf.pkl','wb'))